### Firstly, load the scrapped website data that was saved in json format.


In [68]:
# import all libraries
import json
from openai import OpenAI
import tiktoken

In [69]:
dict = {}
# read json file and save in policy_dict
with open("../output/otago_policies_20250206102803.json") as f:
    dict = json.load(f)

# then print all policy type in the dictionary key
for key in dict.keys():
    print(key)

Guidelines
Plans & Strategies
Policies
Procedures
Regulations
Statutes


In [70]:
# load the api.json file
with open("../api.json") as f:
    api_dict = json.load(f)

# get deepseek-api-key and save to DEEPSEEK_API_KEY
DEEPSEEK_API_KEY = api_dict["deepseek-api-key"]
# get openai-api-key and save to OPENAI_API_KEY
# OPENAI_API_KEY = api_dict['openai-api-key']


In [71]:
# Create count_tokens function which takes in a text and encoding name as input and returns the number of tokens in the text.
def count_tokens(text: str, encoding_name: str = "cl100k_base") -> int:
    """
    Count the number of tokens in a given text using the specified encoding.

    Args:
        text (str): The input text to count tokens for.
        encoding_name (str): The name of the encoding to use. Default is "cl100k_base" (used by GPT-4 and similar models).

    Returns:
        int: The number of tokens in the text.
    """
    # Get the encoding
    encoding = tiktoken.get_encoding(encoding_name)

    # Tokenize the text and count the tokens
    tokens = encoding.encode(text)
    return len(tokens)


In [72]:
# get policy type list and policy
def get_policy_type_details(policy_type: str) -> dict:
    """
    Get the details of a policy type from the Otago University policies.

    Args:
        policy_type (str): The name of the policy type to get details for.

    Returns:
        dict: The details of the policy type.
    """
    # Get the policy type details
    policy_list = []

    for item in dict[policy_type]:
        policy_dict = {}
        policy_dict["name"] = item["name"]
        policy_dict["scope"] = item["scope"]
        policy_dict["content"] = item["content"]
        # append the policy_dict to policy_list
        policy_list.append(policy_dict)
    return policy_list

### Secondly, prepare all questions need to ask LLM model.

In [73]:
q1 = "From below list of JSON data, please list all policies that contain unnecessary content such as cross-references to other policies (there's a space for this in the policy documents, but not in the words of the policy itself)?"

q2 = "From below list of JSON data, please list all policies that contain parts that are actually Procedures, not Policies (e.g. they contain instructions on how to do something, rather than rules or principles)?"

q3 = "From below list of JSON data, please list all policies that contain inconsistencies or structural flaws. For example, if a policy is missing a section that is present in other policies, or if a policy has a section that is not relevant to the policy type?"

q4 = "From below list of JSON data, please list all redundancy or incorrect structure of policy. It could be that there's a better way to structure the policy database, or that some policies are redundant or overlapping?"


### Then design functions to call Deepseek LLM model with api key


In [74]:
def call_llm_about_policy(questions, data, deepseek_model="deepseek-chat"):
    # chec number of questions
    system_content = "I will send you data in any python list format. Each item in the data list contains one policy document record. Each record contains the following fields: 'name', 'scope', and 'content'. Please only answer question once you have read all the data."
    
    client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

    responses = []
    num_question_answered = 0
    for question in questions:
        if num_question_answered == 0:  
            user_question_content = f"{question}\n: {data}"
            messages = [
                {"role": "system", "content": system_content},
                {"role": "user", "content": user_question_content},
            ]
            response = client.chat.completions.create(
                model=deepseek_model, messages=messages
            )
            responses.append(response)
            num_question_answered += 1
            print(
                f"Question {num_question_answered} answered and {response.usage.prompt_tokens} prompt tokens are used as input and {response.usage.completion_tokens} complete tokens to generate the answer."
            )
        else:
            messages.append(response.choices[0].message)
            messages.append({"role": "user", "content": user_question_content})
            response = client.chat.completions.create(
                model=deepseek_model, messages=messages
            )
            responses.append(response)
            num_question_answered += 1
            print(f"Question {num_question_answered} answered.")
            

    # Extract and print the final response
    if len(responses) > 0:
        print("Returning output response:")
        return responses
    else: 
        print("No response to return.")
        return None

In [75]:
guideline_lists = get_policy_type_details("Guidelines")
# show number of policy_list
print(f"There are {len(guideline_lists)} items in the list.")
# show number of policy_list tokens
print(f"There are {count_tokens(str(guideline_lists))} tokens in the policy list.")

processed_document = 0
for policy in guideline_lists:
    response = call_llm_about_policy(q1, policy)
    guideline_lists[processed_document]["q1_response"] = response.message.content
    

print(guideline_lists[0])

guidance_responses_q2 = call_llm_about_policy(q2, guideline_lists)
guidance_responses_q3 = call_llm_about_policy(q3, guideline_lists)
guidance_responses_q4 = call_llm_about_policy(q4, guideline_lists)

There are 54 items in the list.
There are 75481 tokens in the policy list.
Question 1 answered and 1409 prompt tokens are used as input and 89 complete tokens to generate the answer.
Question 2 answered.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
regulation_list = get_policy_type_details("Plans & Strategies")
# show number of policy_list
print(f"There are {len(regulation_list)} items in the list.")
# show number of policy_list tokens
print(f"There are {count_tokens(str(regulation_list))} tokens in the policy list.")

plan_strategies_responses = call_llm_about_policy([q1, q2, q3, q4], regulation_list)
print(plan_strategies_responses[0].choices[0].message.content)

In [ ]:
regulation_list = get_policy_type_details("Policies")
# show number of policy_list
print(f"There are {len(regulation_list)} items in the list.")
# show number of policy_list tokens
print(f"There are {count_tokens(str(regulation_list))} tokens in the policy list.")

policies_response = call_llm_about_policy([q1, q2, q3, q4], regulation_list)
print(policies_response[0].choices[0].message.content)

In [ ]:
regulation_list = get_policy_type_details("Procedures")
# show number of policy_list
print(f"There are {len(regulation_list)} items in the list.")
# show number of policy_list tokens
print(f"There are {count_tokens(str(regulation_list))} tokens in the policy list.")

procedures_responses = call_llm_about_policy([q1, q2, q3, q4], regulation_list)
print(procedures_responses[0].choices[0].message.content)


In [ ]:
regulation_list = get_policy_type_details("Regulations")
# show number of policy_list
print(f"There are {len(regulation_list)} items in the list.")
# show number of policy_list tokens
print(f"There are {count_tokens(str(regulation_list))} tokens in the policy list.")

regulations_responses = call_llm_about_policy([q1, q2, q3, q4], regulation_list)
print(regulations_responses[0].choices[0].message.content)


In [ ]:
regulation_list = get_policy_type_details("Statutes")
# show number of policy_list
print(f"There are {len(regulation_list)} items in the list.")
# show number of policy_list tokens
print(f"There are {count_tokens(str(regulation_list))} tokens in the policy list.")

statues_responses = call_llm_about_policy([q1, q2, q3, q4], regulation_list)
print(statues_responses[0].choices[0].message.content)
